In [1]:
import pandas as pd
import requests
import json
import logging
from bs4 import BeautifulSoup
import time

In [2]:
# GET TOP TOURISM COUNTRIES
df_tourism = pd.read_csv('./data/most_traveled_countries.csv')
# SOURCE: http://worldpopulationreview.com/countries/most-visited-countries/
# df_tourism.head()

top20 = list(df_tourism.sort_values(by='touristArrivals', ascending=False)['name'][:20])
print(top20)

['France', 'United States', 'Spain', 'China', 'Italy', 'United Kingdom', 'Germany', 'Mexico', 'Thailand', 'Turkey', 'Austria', 'Malaysia', 'Hong Kong', 'Greece', 'Russia', 'Japan', 'Canada', 'Portugal', 'Saudi Arabia', 'Poland']


In [3]:
# GET COUNTRIES
country_names = [c_name for c_name in pd.read_csv('./data/country_names.csv')['name']]
# SOURCES: http://worldpopulationreview.com/countries/most-visited-countries/
    # & https://www.travel-advisory.info/api


In [4]:
len(country_names)

178

In [5]:
# GET ALL URLS FOR 20 COUNTRIES
guides = []
articles = []
for country in country_names:
#     print(f'Searching {country}...')
#     for n in range(1, 10):
    try:
        url_base = f'https://www.bemytravelmuse.com/{country}-guide/'
        res_ = requests.get(url_base)
        soup_ = BeautifulSoup(res_.content, 'lxml')
        time.sleep(1)

        guide = {}
        guide['href'] = url_base
        guide['title'] = f'{country} guide'
        guide['country'] = country

        paragraphs = soup_.find_all(attrs={'class': 'so-panel widget widget_sow-editor panel-last-child'})
        ls_words = soup_.find_all(attrs={'class': 'so-widget-sow-editor so-widget-sow-editor-base'})
        all_words = [p.text.replace('\n', '') for p in paragraphs] + [w.text.replace('\n', '') for w in ls_words]
        text = ' '.join(all_words)
        guide['text'] = text

        guides.append(guide)
        
    except:
        print(f'Error while pulling guide data for {country}.')
        
    try:
        
        for link in soup_.find_all('h2', attrs={'class':'entry-title'}):
            article = {}
            article['href'] = link.find('a')['href']
            article['title'] = link.find('a').text.strip()
            article['country'] = country
            articles.append(article)

    except:
        print(f'Error while pulling url data for {country}.')

df_articles = pd.DataFrame(articles)
        
try:    
    content_filed = []
    for url in df_articles['href']:
        res_post = requests.get(url)
        soup_post = BeautifulSoup(res_post.content, 'lxml')

        paragraphs = soup_post.find_all('p', attrs={'class': None}) 

        text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
        content_filed.append(text)

        time.sleep(1)
    df_articles['text'] = content_filed


except:
    print(f'Error while pulling url content.')
        

df = pd.DataFrame(guides)
df = df.append(df_articles, ignore_index=True)
df = df[df['text']!='']

print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (298, 4)


,href,title,country,text
6,https://www.bemytravelmuse.com/Austria-guide/,Austria guide,Austria,"Austria, your are so darn beautiful. I just ca..."
8,https://www.bemytravelmuse.com/Aruba-guide/,Aruba guide,Aruba,Quick Tips'Hello' and 'Thank You' in Papiame...
25,https://www.bemytravelmuse.com/Botswana-guide/,Botswana guide,Botswana,Botswana is my favorite country for wildlife v...
27,https://www.bemytravelmuse.com/Canada-guide/,Canada guide,Canada,"Oh Canada, how I love your gorgeous national a..."
30,https://www.bemytravelmuse.com/Switzerland-guide/,Switzerland guide,Switzerland,Switzerland is the type of place where you mig...


In [6]:
df.tail()

,href,title,country,text
435,https://www.bemytravelmuse.com/what-does-it-co...,What Does It Cost to Travel in South Africa?,South Africa,"In this installment of Tips From the Banker, w..."
436,https://www.bemytravelmuse.com/wine-tasting-in...,"Get Your Vino on In Stellenbosch, South Africa",South Africa,Each place in this world that is known for win...
437,https://www.bemytravelmuse.com/south-africa-ra...,"7 Reasons to Visit South Africa, the Rainbow N...",South Africa,"“Everyone of whatever race, nation or creed wh..."
438,https://www.bemytravelmuse.com/tallest-bungee-...,Taking a Long Walk Off a Short Bridge: The Wor...,South Africa,"To be more correct, the bungee jump at Bloukra..."
439,https://www.bemytravelmuse.com/karoo-south-afr...,The Peaceful Perfection of The Karoo in South ...,South Africa,Have you ever felt truly alone? Not in the sen...


In [7]:
df.to_csv('./data/articles_bemytravelmuse.csv', index=False)